# Example: Cox proportional hazard calculations using the rhino_health SDK

### Load all necessary libraries, including 'rhino_health'

In [ ]:
from getpass import getpass
import rhino_health
import pandas as pd
from rhino_health.lib.metrics import *

### Log in to the Rhino Health Platform

**Note: Replace "USERNAME" with your Rhino Health username.**


In [ ]:
my_username = "my_email@example.com" # Replace this with the email you use to log into Rhino Health

print("Logging In")
session = rhino_health.login(username=my_username, password=getpass())
print("Logged In")

### Run this cell to list all of the projects in your environment:

In [ ]:
projects = session.project.search_for_projects_by_name(name="")
for p in projects:
    print(p.name)

### Load the Project you would like to calculate the metric for by placing the Project's name below
Replace `PROJECT_NAME` with the name of your project

In [ ]:
project = session.project.get_project_by_name("PROJECT_NAME")
if not project:
        raise ValueError("Project not found.")

print("Selected project name:", project.name)

### List your available datasets in your project and then load those you would like to calculate federated percentiles

Run this cell to list all of your available datasets in your project

In [ ]:
all_datasets = session.dataset.search_for_datasets_by_name(name="")
project_datasets = [d for d in all_datasets if d.project and d.project.uid == project.uid]

print(f"\nDatasets in project '{project.name}':")
for d in project_datasets:
    print(f"- {d.name}")

Load the datasets you would like to calculate metrics for by placing the datasets names below,
Replace `DATASET_1` & `DATASET_2` with the name of your datasets

In [ ]:
dataset_uids = [
    project.get_dataset_by_name("DATASET_1"),
    project.get_dataset_by_name("DATASET_2"),
]

The datasets should be in the following format:

In [ ]:
pd.DataFrame({
    'Time': [84.0, 97.0, 91.0, 90.0, 124.0, 97.0],
    'Event': [1, 0, 0, 1, 1, 1],
    'COV1': [0.3, 0.51, 0.12, 0.03, 0.413, 0.3],
    'COV2': [5.3, 1.51, 1.8, 0.03, 13, 0.3]
})

`Time`: The time-to-event or censoring,

`Event`: The event indicator (1 if the event occurred, 0 if censored),

`COV1` and `COV2`: are the Covariates that may influence the time-to-event.

In [ ]:
# Set the time and event variables
time_variable = "Time"
event_variable = "Event"
covariates = ["COV1", "COV2"]

# Create a Cox instance, use the mean of the local betas of the two sites as the initial beta
metric_configuration = Cox(time_variable=time_variable, event_variable=event_variable, covariates=covariates, initial_beta="mean", max_iterations=50)

# Retrieve results for your project and datasets
results = project.aggregate_dataset_metric(dataset_uids=[str(dataset.uid) for dataset in dataset_uids], metric_configuration=metric_configuration)